**Bioinformatique : Rendu_6** <Br>
Hajar Hajji

## La méthode UPGMA

### Etape 1 : calcul des distances entre toutes les paires de séquences

#### Etape 1-1 : lecture de l’alignement multiple

In [234]:
def lecture_sequences(fichier_sequences):
    """
    fonction permettant de lire des séquences contenues dans un fichier
    au format fasta et de les stocker dans un dictionnaire (clé = nom de 
    la séquence - valeur = séquence) qui est renvoyé (None en cas d'erreur)
    """  

    # ouverture du fichier
    fichier_entree = open(fichier_sequences, 'r')
            
    # lecture de la 1ère ligne contenant le descriptif
    ligne = fichier_entree.readline()
            
    # vérification du format du fichier
    if ligne[0] != '>':
        print('Votre fichier n\'est pas au format fasta')
        return None
    
    # dictionnaire des séquences
    sequences = dict()
    
    # longueur des séquences
    longueur_ref = 0
    
    continuer = True
    # tant qu'il reste des séquences
    while continuer:
        
        # lecture du nom de la séquence
        nom_sequence = ligne[1:-1]
        
        # lecture des lignes suivantes contenant la séquence
        sequence = ''
        ligne = fichier_entree.readline()
        while ligne != '' and ligne[0] != '>':
            # écriture en lettres majuscules
            ligne = ligne.upper()
            # vérification des caractères
            caracteres_autorises = 'ACDEFGHIKLMNPQRSTVWY'
            for c in ligne:
                if c in caracteres_autorises:
                    sequence += c
                else:
                    if c != '\n':
                        print('La séquence %s contient un caractère inattendu : %s'
                              % (nom_sequence,c))
                        return None                
            ligne = fichier_entree.readline()
        
        #ajout de la séquence au dictionnaire
        sequences[nom_sequence] = sequence
        
        #si on a atteint la fin du fichier
        if ligne == '':
            continuer = False
            longueur_ref = len(sequence)
    
    fichier_entree.close()
    
    #vérification de la longueur des sequences
    for nom in sequences:
        if len(sequences[nom]) != longueur_ref:
            print('Vos séquences n\'ont pas toutes la même longueur')
            return None            
            
    return sequences

In [235]:
nom_fichier="alignement_multiple_cytochrome-b.fasta"
sequences=lecture_sequences(nom_fichier)
sequences

{'Poulet': 'VCLMSQILTGLLLAMHYTDTLAFSSVAHTRNVQYGWLIRNLHANGASFFICIFLHIGRGLYYGSYLYKETWNTGVILLLTLMATAFVGYVLPWGQMSFWGATVITNLFSAIPYIGHTLVEWAWGGFSVDNPTLTRFFALHFLLPFAIAGITIIHLTFLHESGSNNPLGISS',
 'Vautour': 'ICLMSQILTGLLLAMHYTDTLAFSSVAHTRNVQYGWLIRNLHANGASFFICIYLHIGRGLYYGSYLYKETWNTGVILLLTLMASAFVGYVLPWGQMSFWGATVITNLFSAIPYIGQTLVEWAWGGFSVDNPTLTRFFALHFLLPFMITGLTLIHLTFLHESGSNNPLGIVS',
 'Tortue': 'VCLMSNILTGIFLAMHYSDIMAFSSIAHIRDVQYGWLIRNMHANGASFFMCIYLHIGRGIYYGSYLYKETWNTGIILLLLVMATAFVGYVLPWGQMSFWGATVITNLLSAIPYIGNTLVQWIWGGFSVDNATLTRFFTFHFLLPFAITGLTAVHLLFLHETGSNNPMGLNS',
 'Homme': 'ACLILQITTGLFLAMHYSDATAFSSIAHIRDVNYGWIIRYLHANGASFFICLFLHIGRGLYYGSFLYSETWNIGIILLLATMATAFMGYVLPWGQMSFWGATVITNLLSAIPYIGTDLVQWIWGGYSVDSPTLTRFFTFHFILPFIIAALATLHLLFLHETGSNNPLGITS',
 'Souris': 'VCLMVQITTGLFLAMHYTDTTAFSSVTHIRDVNYGWLIRYMHANGASFFICLFLHVGRGLYYGSYTFMETWNIGVLLLFAVMATAFMGYVLPWGQMSFWGATVITNLLSAIPYIGTTLVEWIWGGFSVDKATLTRFFAFHFILPFIIAALAIVHLLFLHETGSNNPTGLNS',
 'Xenope': 'VCLVTQIVTGLFLAMHYTDTMAFSSVAHIFDVNYGLLIRNLHANGLSFFICIYLHIGR

In [236]:
sequences["Poulet"] #accéder à la séquence par la clé qui est le nom de la séquence

'VCLMSQILTGLLLAMHYTDTLAFSSVAHTRNVQYGWLIRNLHANGASFFICIFLHIGRGLYYGSYLYKETWNTGVILLLTLMATAFVGYVLPWGQMSFWGATVITNLFSAIPYIGHTLVEWAWGGFSVDNPTLTRFFALHFLLPFAIAGITIIHLTFLHESGSNNPLGISS'

#### Etape 1-2 : calcul des distances

In [237]:
def calcul_matrice_initiale(sequences):
    
    """
    fonction prenant en argument un dictionnaire de séquences et renvoyant 
    un dictionnaire contenant les distances observées entre toutes les paires de
    séquences (clé = tuple contenant le nom de deux séquences ; valeur = 
    distance entre ces deux séquences)
    """
    
    distances={}
    for nom_sequence1,sequence1 in sequences.items():
        for nom_sequence2,sequence2 in sequences.items():
            if nom_sequence1==nom_sequence2:
                distances[(nom_sequence1,nom_sequence2)]=0
            else:
                difference=0
                for ch1,ch2 in zip(sequence1,sequence2):
                    if ch1!=ch2:
                        difference+=1
                distance=difference/len(sequence1)
                distances[(nom_sequence1,nom_sequence2)]=distances[(nom_sequence2,nom_sequence1)]=distance
        
    return distances

In [238]:
def calcul_matrice_initiale_2(sequences):
    
    """
    2ème manière de faire
    """
    
    distances=dict()
    
    #parcourir chaque paire de sequences
    for key1 in sequences:
        for key2 in sequences:
            if key1==key2:
                distances[(key1,key2)]=0
                
        sequence1,sequence2=sequences[key1],sequences[key2]
          
        #calculer le nombre de positions où les caractères de sequence1 et sequence2 sont différents
        #distance observée entre les deux séquences -> pourcentage de différences entre les 2 séquences
        distance=sum(a != b for a, b in zip(sequence1, sequence2)) / len(sequence1)
        
        distances[(key1,key2)]=distances[(key2,key1)]=distance
            
    return distances

In [239]:
# ---------------        fonctions auxiliaires        --------------- #
# ---------------      nécessaires à l'affichage      --------------- #


def liste_noms_sequences_affichage(noms):
    """
    fonction renvoyant la liste des noms des séquences écrits sous une forme
    compatible avec l'affichage de la matrice de distances
    """
    
    noms_affichage = dict()
    
    for nom in noms:
        nb_maj = 0
        maj = ''
        for lettre in nom:
            if lettre.isupper():
                nb_maj += 1
                maj += lettre
        if nb_maj == 1:
            noms_affichage[nom] = nom
        else:
            noms_affichage[nom] = maj
    
    return noms_affichage

## -------------------------------------------------------------------------------- ##
## -------------------------------------------------------------------------------- ##

def affichage_distances(distances):
    """
    fonction permettant un affichage formaté de la matrice de distances
    """
    
    #liste des noms des séquences
    noms = [paire[0] for paire in list(distances.keys())]
    noms = list(set(noms))
    
    #liste des noms des séquences pour l'affichage
    noms_affichage = liste_noms_sequences_affichage(noms)
    
    #première ligne contenant les noms des séquences
    res = ' '*10
    for nom in noms:
        res = res + noms_affichage[nom][:6]
        if len(noms_affichage[nom]) > 6:
            res = res + '.' + ' '*3
        else:
            res = res + ' '*(6-len(noms_affichage[nom])+4)
    res = res + '\n'
    #autres lignes
    for nom1 in noms:
        #nom de la séquence
        res = res + noms_affichage[nom1][:6]
        if len(noms_affichage[nom1]) > 6:
            res = res + '.' + ' '*3
        else:
            res = res + ' '*(6-len(noms_affichage[nom1])+4)
        #distances
        for nom2 in noms:
            res = res + '%.4f' % (round(distances[(nom1, nom2)],4),) + ' '*4
        #passage à la ligne
        res = res + '\n'
    
    return res

In [240]:
distances=calcul_matrice_initiale(sequences)
print("Matrice des distances :")
print()
print(affichage_distances(distances))

Matrice des distances :

          Homme     Tortue    Truite    Chondr.   Xenope    Souris    Poulet    Vautou.   
Homme     0.0000    0.2164    0.2281    0.2398    0.2281    0.1696    0.2398    0.2515    
Tortue    0.2164    0.0000    0.2047    0.1637    0.1871    0.2105    0.1871    0.1871    
Truite    0.2281    0.2047    0.0000    0.1404    0.1813    0.2222    0.2281    0.2339    
Chondr.   0.2398    0.1637    0.1404    0.0000    0.1520    0.1871    0.2164    0.2105    
Xenope    0.2281    0.1871    0.1813    0.1520    0.0000    0.1930    0.2105    0.2281    
Souris    0.1696    0.2105    0.2222    0.1871    0.1930    0.0000    0.2222    0.2456    
Poulet    0.2398    0.1871    0.2281    0.2164    0.2105    0.2222    0.0000    0.0526    
Vautou.   0.2515    0.1871    0.2339    0.2105    0.2281    0.2456    0.0526    0.0000    



### Etape 2 : la méthode UPGMA


In [241]:
def minimum_matrice(distances):
    
    """
    renverra le nom du couple de séquences séparé par la plus petite distance 
    """
    
    min_distance=float('inf') #initialiser à l'inf
    min_paire=None
    
    for paire, distance in distances.items():
        if paire[0]==paire[1]: #diagonale exclue
            continue
        if distance < min_distance:
            min_distance=distance
            min_paire=paire
                
    return min_paire,min_distance

In [242]:
min_paire,min_dist=minimum_matrice(distances)
min_paire,min_dist

(('Poulet', 'Vautour'), 0.05263157894736842)

In [243]:
def construction_arbre(arbres_partiels, nom1, nom2):
    """
    fonction prenant en argument un dictionnaire des arbres partiels déjà
    construits et le nom du couple de "séquences" à regrouper et retournant  
    les nouveaux arbres partiels obtenus après ce regroupement
    """
    
    #Case 1 : si les séquences nom1 et nom2 ne sont pas encore dans des arbres partiels
    if nom1 not in arbres_partiels and nom2 not in arbres_partiels:
        arbres_partiels[nom1+nom2] = '(' + nom1 + ',' + nom2 + ')'
    
    #Case 2 : si la séquence nom2 est déjà dans un arbre partiel
    elif nom1 not in arbres_partiels:
        arbres_partiels[nom1+nom2] = '(' + nom1 + ',' + arbres_partiels[nom2] + ')'
        arbres_partiels.pop(nom2)
    
    #Case 3 : si la séquence nom1 est déjà dans un arbre partiel
    elif nom2 not in arbres_partiels:
        arbres_partiels[nom1+nom2] = '(' + arbres_partiels[nom1] + ',' + nom2 + ')'
        arbres_partiels.pop(nom1)
    
    #Case 4 : si les séquences nom1 et nom2 sont déjà dans des arbres partiels
    else:
        arbres_partiels[nom1+nom2] = '(' + arbres_partiels[nom1] + ',' + arbres_partiels[nom2] + ')'
        arbres_partiels.pop(nom1)
        arbres_partiels.pop(nom2)
    
    return arbres_partiels

In [244]:
# ---------------      Exemples de test      --------------- #

#Case 1 : si les séquences nom1 et nom2 ne sont pas encore dans des arbres partiels
arbres_partiels = {'A': 'A', 'B': 'B', 'C': 'C', 'D': 'D'}
print(construction_arbre(arbres_partiels,'V','G'))
## -------------------------------------------------------------------------------- ##
#Case 2 : si la séquence nom2 est déjà dans un arbre partiel
print(construction_arbre(arbres_partiels,'Z','C'))
## -------------------------------------------------------------------------------- ##
#Case 3 : si la séquence nom1 est déjà dans un arbre partiel
print(construction_arbre(arbres_partiels,'D','H'))
## -------------------------------------------------------------------------------- ##
#Case 4 : si les séquences nom1 et nom2 sont déjà dans des arbres partiels
print(construction_arbre(arbres_partiels,'C','B'))

{'A': 'A', 'B': 'B', 'C': 'C', 'D': 'D', 'VG': '(V,G)'}
{'A': 'A', 'B': 'B', 'D': 'D', 'VG': '(V,G)', 'ZC': '(Z,C)'}
{'A': 'A', 'B': 'B', 'VG': '(V,G)', 'ZC': '(Z,C)', 'DH': '(D,H)'}
{'A': 'A', 'VG': '(V,G)', 'ZC': '(Z,C)', 'DH': '(D,H)', 'CB': '(C,B)'}


In [245]:
# def construction_arbre_2(arbres_partiels,nom_sequence1,nom_sequence2):
#     """
#     fonction prenant en argument un dictionnaire des arbres partiels déjà
#     construits et le nom du couple de "séquences" à regrouper et retournant  
#     les nouveaux arbres partiels obtenus après ce regroupement
#     """
#     #arbres_partiels = {'A': 'A', 'B': 'B', 'C': 'C', 'D': 'D'}
    
#     #arbres_partiels = construction_arbre(arbres_partiels, 'A', 'B')
#     # => arbres_partiels = {'C': 'C', 'D': 'D', 'AB': '(A,B)'}
    
#     #arbres_partiels = construction_arbre(arbres_partiels, 'A', 'C')
#     # => arbres_partiels = {'D': 'D', 'AC': '(A,C)', 'AB': '(A,B)'}
    
#     nouveau_nom=nom_sequence1+nom_sequence2
    
#     #si les 2 séquences ne sont pas encore dans des arbres partiels
#     if nom_sequence1 and nom_sequence2 not in arbres_partiels.keys():
#         arbres_partiels[nouveau_nom]='('+nom_sequence1+','+nom_sequence2+')'
    
#     #si les 2 séquences sont dans des arbres partiels
#     elif nom_sequence1 and nom_sequence2 in arbres_partiels.keys():
#         arbres_partiels[nouveau_nom]='('+arbres_partiels[nom_sequence1]+','+arbres_partiels[nom_sequence2]+')'
#         arbres_partiels.pop(nom_sequence1)
#         arbres_partiels.pop(nom_sequence2)
    
#     #si le nom de la 1ère séquence est déjà dans des arbres partiels
#     elif nom_sequence1 in arbres_partiels.keys() and nom_sequence2 not in arbres_partiels.keys():
#         arbres_partiels[nouveau_nom]='('+arbres_partiels[nom_sequence1]+','+nom_sequence2+')'
#         arbres_partiels.pop(nom_sequence1)
        
#     #si le nom de la 2ème séquence est déjà dans des arbres partiels
#     elif nom_sequence2 in arbres_partiels.keys() and nom_sequence1 not in arbres_partiels.keys():
#         arbres_partiels[nouveau_nom]='('+nom_sequence1+','+arbres_partiels[nom_sequence2]+')'
#         arbres_partiels.pop(nom_sequence2)
        
#     return arbres_partiels

In [246]:
def reduction_matrice(distances,nom_sequence1,nom_sequence2):
    
    """
    fonction réduisant le dictionnaire des distances en fusionnant les
    "séquences" de noms nom_sequence1 et nom_sequence2 en une seule "séquence" ayant pour 
    nom la concaténation de leurs deux noms
    """
    
    noms_sequences=[paire[0] for paire in list(distances.keys())]
    noms_sequences=list(set(noms_sequences))
    
    nouveau_nom=nom_sequence1+nom_sequence2
    
    distances[(nouveau_nom,nouveau_nom)]=0
    
    for nom_sequence in noms_sequences:
        if nom_sequence not in [nom_sequence1,nom_sequence2]:
            distances[(nouveau_nom,nom_sequence)]=distances[(nom_sequence,nouveau_nom)]\
            =(distances[(nom_sequence,nom_sequence1)]+distances[(nom_sequence,nom_sequence2)]) / 2
    
    #enlever les anciennes distances
    for nom_sequence_1 in noms_sequences:
        for nom_sequence_2 in noms_sequences:
            if nom_sequence_1 in [nom_sequence1,nom_sequence2] or nom_sequence_2 in [nom_sequence1,nom_sequence2]:
                if (nom_sequence_1,nom_sequence_2) in distances or (nom_sequence_1,nom_sequence_2) in distances:
                    if nom_sequence_1==nom_sequence_2:
                        distances.pop((nom_sequence_1,nom_sequence_2))
                    else:
                        distances.pop((nom_sequence_1,nom_sequence_2))
                        distances.pop((nom_sequence_2,nom_sequence_1))
        
    return distances

In [247]:
def UPGMA(distances):
    
    """
    fonction prenant en argument un dictionnaire des distances deux à deux
    d'un ensemble de séquences et renvoyant l'arbre obtenu à partir de ces
    distances avec la méthode UPGMA
    """
    
    arbres_partiels={}
    
    #critère d'arrêt : quand la matrice aurait un seul élément
    while len(distances) > 1: 
        
        #chercher la paire avec min distances (diagonale exclue...)
        min_paire,min_distance=minimum_matrice(distances)
        nom_sequence_1,nom_sequence_2=min_paire
        print("Distance minimale :")
        print(f"({min_paire[0]},{min_paire[1]}) : {min_distance}")
        print()
    
        arbres_partiels=construction_arbre(arbres_partiels,nom_sequence_1,nom_sequence_2)
        print("Arbres partiels : ")
        for arbre in arbres_partiels:
            print(arbres_partiels[arbre])
        print()
        
        #mettre à jour la matrice distances
        print("Nouvelle matrice des distances :")
        print()
        reduction_matrice(distances,nom_sequence_1,nom_sequence_2)
        print(affichage_distances(distances))
        
        print("-----------------------------------------------------------------------------------")
        print()
    
    print("/// L'exécution s'est terminée ///")
    print()
    print("Arbre final (format newick) :")
    print(arbres_partiels)
    
    for arbre in arbres_partiels:
        return arbres_partiels[arbre]+";"

In [248]:
# ---------------         programme principal         --------------- #

nom_fichier="alignement_multiple_cytochrome-b.fasta"
sequences=lecture_sequences(nom_fichier)

distances=calcul_matrice_initiale(sequences)
print("Matrice initiale de distances :")
print()
print(affichage_distances(distances))

#construire l'arbre avec la méthode UPGMA
arbres_partiels=UPGMA(distances)

#Sauvegarder dans un fichier txt externe
monfichier=open("arbre_final_UPGMA.txt","w")
monfichier.write(arbres_partiels)
monfichier.close()
print()
print("Sauvegardé avec succès !")

Matrice initiale de distances :

          Homme     Tortue    Truite    Chondr.   Xenope    Souris    Poulet    Vautou.   
Homme     0.0000    0.2164    0.2281    0.2398    0.2281    0.1696    0.2398    0.2515    
Tortue    0.2164    0.0000    0.2047    0.1637    0.1871    0.2105    0.1871    0.1871    
Truite    0.2281    0.2047    0.0000    0.1404    0.1813    0.2222    0.2281    0.2339    
Chondr.   0.2398    0.1637    0.1404    0.0000    0.1520    0.1871    0.2164    0.2105    
Xenope    0.2281    0.1871    0.1813    0.1520    0.0000    0.1930    0.2105    0.2281    
Souris    0.1696    0.2105    0.2222    0.1871    0.1930    0.0000    0.2222    0.2456    
Poulet    0.2398    0.1871    0.2281    0.2164    0.2105    0.2222    0.0000    0.0526    
Vautou.   0.2515    0.1871    0.2339    0.2105    0.2281    0.2456    0.0526    0.0000    

Distance minimale :
(Poulet,Vautour) : 0.05263157894736842

Arbres partiels : 
(Poulet,Vautour)

Nouvelle matrice des distances :

          Homme 